In [2]:
!python -m pip install pytesseract
!python -m pip install opencv-python

In [3]:
pip install pymupdf pillow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' # replace this path with the path to your tesseract installation

In [2]:
import fitz
from PIL import Image
import pandas as pd
from pandas import DataFrame as df
from io import StringIO
from pathlib import Path
import csv

In [18]:
def pdf_to_png(pdf_path, output_folder):
    '''Chop a given PDF into individual pages, then convert each PDF into an image (saved to the pngs folder). Convert OCR data about each page into a .tsv DataFrame.
        This function requires that you have a folder in the same level as your pdf for outputs, and then two folders within that folder titled 'pngs' and 'tsv_data'.
        For example:
            your pdf
            output folder
                tsv_data
                pngs'''
    
    open_pdf = fitz.open(pdf_path)

    sum_string = ''

    for page_num in range(open_pdf.page_count): # iterate through individual pages
        page = open_pdf[page_num]

        img = page.get_pixmap()
        tsv_total = pd.DataFrame()

        # make the image
        pil_img = Image.frombytes("RGB", [img.width, img.height], img.samples) # convert to PIL Image
        # improve resolution
        scale_factor = 3 # try changing this to improve resolution
        new_size = (img.width * scale_factor, img.height * scale_factor)
        resize = pil_img.resize(new_size, Image.LANCZOS)
        

        # process image into tsv and clean some values
        png_to_data = pytesseract.image_to_data(resize, config = r'--psm 6') 
        data_to_tsv = StringIO(png_to_data)
        data_read = pd.read_csv(data_to_tsv, sep='\t', quoting=csv.QUOTE_NONE)
        tsv_clean = data_read[['line_num', 'word_num', 'left', 'top', 'text', 'conf']]
        tsv_total = pd.concat([tsv_total, tsv_clean])

        img_to_string = pytesseract.image_to_string(resize, config = r'--psm 6')
        sum_string += img_to_string

        # save to files
        output_csv_path = Path(f"{output_folder}/tsv_data/page_{page_num + 1}.tsv")
        #tsv_clean.to_csv(output_csv_path, sep = '\t', index = False)

        output_png_path = Path(f"{output_folder}/pngs/page_{page_num + 1}.png")
        resize.save(output_png_path)

    output_tsv_total = Path(f"{output_folder}/tsv_data.tsv")
    tsv_total.to_csv(output_tsv_total, sep= '\t', index=False)
    print(sum_string)
    output_str_path = Path(f"{output_folder}/str_data.txt")
    with open(output_str_path, 'w') as file:
        file.write(sum_string)

In [19]:
# replace these with the paths to your files
pdf_path = r'TOCs\NAAL TOCs\1985 Baym\clean_1985_Baym_The_Norton_anthology_of_American_vol_1.pdf'
output_folder = r'TOCs\NAAL TOCs\1985 Baym\processed'

pdf_to_png(pdf_path, output_folder)

Contents
PREFACE TO THE SECOND EDITION xxvii
ACKNOWLEDGMENTS XXXill
Early American Literature 1620-1820 :
JOHN SMITH (1580-1631) 18
The Gencral History of Virginia 12
The Third Book 12
Chapter 1. 12
Chapter 2. What Happened till the First Supply 16
THOMAS MORTON (c. 1579-1647) 26
New English Canaan 27
The Second Book 27
Chapter I. The General Survey of the Country 27
The Third Book 28
[The Incident at Merry Mount} 28
JOHN WINTHROP (1588-1649) 36
A Model of Christian Charity 37
The Journal of John Winthrop so
WILLIAM BRADFORD (i: 590-1657) 39
Of Plymouth Plantation — 6
Book I, Chapter I. [The Separatist Interpretation of the
Reformation in England, 1550-1607] 60
Book I, Chapter VII. Of Their Departure from Leyden
[Mr. Robinson's Letter] 62
Book 1, Chapter 1X. Of Their Voyage and How They
Passed the Sea; and of Their Safe Arrival at
Cape Cod 68
Book I, Chapter X, Showing How They Sought Ont |
a Place of Habitation; and What Befell Them
Thereabout 71
vit
meee
ee
viii * Contents
Book 11, C